In [175]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from numba import njit
import networkx as nx

In [176]:
def calcular_variancia_ruido(Eb_N0_dB):
    Eb_N0 = 10**(Eb_N0_dB / 10)
    N0 = 1 / Eb_N0
    sigma2 = N0 / 2
    return sigma2

In [177]:
variancia_ruido = calcular_variancia_ruido(5)
print(f"Variância do ruído: {variancia_ruido}")

Variância do ruído: 0.15811388300841897


In [178]:
def gerar_simbolos_bpsk(num_bits):
    return np.random.choice([-1, 1], num_bits)

In [179]:
simbolos_bpsk = gerar_simbolos_bpsk(10)
print(f"Símbolos BPSK gerados: {simbolos_bpsk}")

Símbolos BPSK gerados: [-1  1  1 -1  1  1 -1  1 -1  1]


In [180]:
def calcular_valor_recebido(simbolos, Eb_N0_dB):
    sigma2 = calcular_variancia_ruido(Eb_N0_dB)
    sigma = np.sqrt(sigma2)
    ruido = sigma * np.random.randn(len(simbolos))
    r = simbolos + ruido
    return r

In [181]:
valores_recebidos = calcular_valor_recebido(simbolos_bpsk, 3)
print(f"Valores recebidos: {valores_recebidos}")

Valores recebidos: [-1.05714712  0.85671148  0.82926633 -0.05692977  0.27410151  1.05512171
 -1.19166915  0.98184066 -1.23179209  1.37347742]


In [182]:
def calcular_valores_llr(r, sigma2):
    llr = 2 * r / sigma2
    return llr

In [183]:
Lc = calcular_valores_llr(valores_recebidos, variancia_ruido)
print(f"Valores LLR: {Lc}")

Valores LLR: [-13.37197085  10.83663827  10.48948157  -0.720111     3.46714028
  13.34635121 -15.07355494  12.41941113 -15.58107443  17.37326788]


In [184]:
@njit
def set_params(N: int, dv: int, dc: int) -> tuple[int, int]:
    if (N  % dc != 0):
        raise ValueError("N must be divisible by dc")
    else:
        K = int(N / dc)
        M = int(K * dv)
        return (K, M)

In [185]:
N = 10
dv = 3
dc = 5
K, M = set_params(N, dv, dc)
print(f"K: {K}; M: {M}")

K: 2; M: 6


In [186]:
@njit
def generate_LDPC_matrix(N, dv, dc, K, M):
    H1 = np.zeros((K, N), dtype=np.int32)
    for i in range(len(H1)):
        H1[i][i * dc:(i + 1) * dc] = 1

    H = H1

    for _ in range(dv - 1):
        perm = np.random.permutation(N)
        H2 = H1[:, perm]
        H = np.concatenate((H, H2), axis=0)

    return H

In [187]:
H = generate_LDPC_matrix(N, dv, dc, K, M)
print(f"Matriz LDPC gerada: \n{H}")

Matriz LDPC gerada: 
[[1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1]
 [1 1 1 0 1 0 0 0 0 1]
 [0 0 0 1 0 1 1 1 1 0]
 [1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1]]


In [188]:
def inicializar_mensagens(H):
    num_cnodes, num_vnodes = H.shape
    Le = np.zeros((num_cnodes, num_vnodes))  # Mensagens dos c-nodes para os v-nodes
    Lr = np.zeros((num_cnodes, num_vnodes))  # Mensagens dos v-nodes para os c-nodes
    return Le, Lr

In [189]:
Le, Lr = inicializar_mensagens(H)
print(f"Mensagens Le iniciais: {Le}")
print(f"Mensagens Lr iniciais: {Lr}")

Mensagens Le iniciais: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Mensagens Lr iniciais: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [190]:
def atualizar_mensagens_vnodes(Lr, Lc, Le, H):
    num_cnodes, num_vnodes = H.shape
    for i in range(num_vnodes):
        for j in range(num_cnodes):
            if H[j, i] == 1:
                Lr[j, i] = Lc[i] + np.sum(Le[:, i]) - Le[j, i]
    return Lr

In [191]:
Lr = atualizar_mensagens_vnodes(Lr, Lc, Le, H)
print(f"Mensagens Lr atualizadas: {Lr}")

Mensagens Lr atualizadas: [[-13.37197085  10.83663827  10.48948157  -0.720111     3.46714028
    0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.
   13.34635121 -15.07355494  12.41941113 -15.58107443  17.37326788]
 [-13.37197085  10.83663827  10.48948157   0.           3.46714028
    0.           0.           0.           0.          17.37326788]
 [  0.           0.           0.          -0.720111     0.
   13.34635121 -15.07355494  12.41941113 -15.58107443   0.        ]
 [-13.37197085  10.83663827  10.48948157  -0.720111     3.46714028
    0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.
   13.34635121 -15.07355494  12.41941113 -15.58107443  17.37326788]]


In [192]:
def atualizar_mensagens_cnodes(Le, Lr, H):
    num_cnodes, num_vnodes = H.shape
    for j in range(num_cnodes):
        for i in range(num_vnodes):
            if H[j, i] == 1:
                # Calcula o produto dos sinais
                product_sign = 1
                min_value = float('inf')
                for k in range(num_vnodes):
                    if k != i and H[j, k] == 1:
                        product_sign *= np.sign(Lr[j, k])
                        min_value = min(min_value, abs(Lr[j, k]))
                # Calcula a mensagem de saída usando a equação 11
                Le[j, i] = product_sign * min_value
    return Le

In [193]:
Le = atualizar_mensagens_cnodes(Le, Lr, H)
print(f"Mensagens Le atualizadas: {Le}")

Mensagens Le atualizadas: [[ -0.720111     0.720111     0.720111    -3.46714028   0.720111
    0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.
   12.41941113 -12.41941113  13.34635121 -12.41941113  12.41941113]
 [  3.46714028  -3.46714028  -3.46714028   0.         -10.48948157
    0.           0.           0.           0.          -3.46714028]
 [  0.           0.           0.          12.41941113   0.
   -0.720111     0.720111    -0.720111     0.720111     0.        ]
 [ -0.720111     0.720111     0.720111    -3.46714028   0.720111
    0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.
   12.41941113 -12.41941113  13.34635121 -12.41941113  12.41941113]]


In [194]:
def iteracao(Lc, Le, Lr, H):
    # Atualizar mensagens dos v-nodes
    Lr = atualizar_mensagens_vnodes(Lr, Lc, Le, H)
    
    # Atualizar mensagens dos c-nodes
    Le = atualizar_mensagens_cnodes(Le, Lr, H)
    
    # Critério de parada: verificar se todas as equações de paridade são satisfeitas
    decision = np.sign(Lc + np.sum(Le, axis=0))
    return decision

In [195]:
n = 0

In [196]:
n += 1
decision = iteracao(Lc, Le, Lr, H)
print(f"Iteração n: {n}")
print(f"Mensagens Le atualizada: {Le}")
print(f"Mensagens Lr atualizada: {Lr}")
np.all(np.mod(np.dot(H, decision), 2) == 0)

Iteração n: 1
Mensagens Le atualizada: [[ -6.3022303    6.3022303    6.3022303    6.3022303   -7.74245229
    0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.
   25.04565135 -25.04565135  25.04565135 -25.04565135  25.04565135]
 [  4.90736227  -4.90736227  -4.90736227   0.         -11.92970357
    0.           0.           0.           0.          -4.90736227]
 [  0.           0.           0.          38.18517348   0.
   -7.65439155   7.65439155  -7.65439155   7.65439155   0.        ]
 [ -6.3022303    6.3022303    6.3022303    6.3022303   -7.74245229
    0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.
   25.04565135 -25.04565135  25.04565135 -25.04565135  25.04565135]]
Mensagens Lr atualizada: [[-10.62494157   8.08960899   7.74245229   8.23215986  -6.3022303
    0.           0.           0.           0.           0.        ]
 [  0.         

False

In [197]:
def ldpc_decode(Lc, H, max_iter=100):
    Le, Lr = inicializar_mensagens(H)
    n = 0

    for iteration in range(max_iter):
        n += 1
        decision = iteracao(Lc, Le, Lr, H)
        if np.all(np.mod(np.dot(H, decision), 2) == 0):
            break
    
    print(f"{n} iterações")
    # Decisão final
    Lf = Lc + np.sum(Le, axis=0)
    decoded_bits = np.sign(Lf)
    return decoded_bits

In [198]:
bits_decodificados = ldpc_decode(Lc, H)
print(f"Bits gerados:       {simbolos_bpsk}")
print(f"Bits decodificados: {bits_decodificados}")


100 iterações
Bits gerados:       [-1  1  1 -1  1  1 -1  1 -1  1]
Bits decodificados: [-1.  1.  1.  1. -1.  1. -1.  1. -1.  1.]


In [ ]:
def calcular_ber(bits_originais, bits_decodificados):

    erros = np.count_nonzero(bits_originais != bits_decodificados)
    ber = erros / len(bits_originais)
    return ber

In [ ]:
ber = calcular_ber(simbolos_bpsk, bits_decodificados)
print(f"Ber: {ber}")

In [ ]:
def simular_desempenho_ldpc(H, num_bits, Eb_N0_dB_range, max_iter=50):
    ber_results = []
    
    for Eb_N0_dB in Eb_N0_dB_range:
        # Gerar símbolos BPSK
        simbolos = gerar_simbolos_bpsk(num_bits)
        
        # Calcular valores recebidos com ruído
        r = calcular_valor_recebido(simbolos, Eb_N0_dB)
        
        # Calcular valores LLR
        sigma2 = calcular_variancia_ruido(Eb_N0_dB)
        Lc = calcular_valores_llr(r, sigma2)
        
        # Decodificar
        bits_decodificados = ldpc_decode(Lc, H, max_iter=max_iter)
        
        # Calcular BER
        ber = calcular_ber(simbolos, bits_decodificados)
        ber_results.append(ber)
        
    return ber_results

In [ ]:
# Definindo os parâmetros para simulação
num_bits = 999
Eb_N0_dB_range = np.arange(0, 5.5, 0.5)

In [ ]:
# Gerando a matriz LDPC
N = num_bits
dv = 4
dc = 9
K, M = set_params(N, dv, dc)
H = generate_LDPC_matrix(N, dv, dc, K, M)

In [ ]:
# Simulando o desempenho
ber_results = simular_desempenho_ldpc(H, num_bits, Eb_N0_dB_range)

In [ ]:
def plotar_ber(Eb_N0_dB_range, ber_results):
    plt.figure(figsize=(10, 6))
    plt.plot(Eb_N0_dB_range, ber_results, marker='o', linestyle='-', color='b')
    plt.yscale('log')
    plt.xlabel('Eb/N0 (dB)')
    plt.ylabel('Taxa de Erro de Bit (BER)')
    plt.title('Desempenho do Código LDPC')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

In [ ]:
# Plotar o gráfico
plotar_ber(Eb_N0_dB_range, ber_results)

In [ ]:
def encontrar_menor_eb_n0(ber_results, Eb_N0_dB_range, threshold=1e-4):
    for Eb_N0_dB, ber in zip(Eb_N0_dB_range, ber_results):
        if ber < threshold:
            return Eb_N0_dB
    return None

# Encontrar o menor valor de Eb/N0 para BER < 10^-4
menor_eb_n0 = encontrar_menor_eb_n0(ber_results, Eb_N0_dB_range)
print(f"Menor valor de Eb/N0 para BER < 10^-4: {menor_eb_n0} dB")

In [ ]:
import csv

def gerar_grafo_csv(H, filename='grafo_ldpc.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        for row in H:
            indices = np.where(row == 1)[0]
            writer.writerow(indices)

# Exemplo de matriz H para N ≈ 1000
H = generate_LDPC_matrix(N, dv, dc, K, M)

# Gerar arquivo CSV
gerar_grafo_csv(H)